In [1]:
import pandas as pd
from xml.dom import minidom
import xml.etree.ElementTree as ET
import re

# Data obtention

In [2]:
# This function reads the data and returns pandas dataframes
def extract():
    pizzas = pd.read_csv('ORIGINALS/pizzas.csv')
    pizza_types = pd.read_csv('ORIGINALS/pizza_types.csv', encoding='latin-1')
    orders = pd.read_csv('ORIGINALS/orders_2015.csv')
    order_details = pd.read_csv('ORIGINALS/order_details_2015.csv')
    data_dictionary = pd.read_csv('ORIGINALS/data_dictionary.csv')
    return pizzas, pizza_types, orders, order_details, data_dictionary

# Data analysis

In [3]:
data = extract()
df_name = ['pizzas', 'pizza_types', 'orders', 'order_details', 'data_dictionary']
for idx, df in enumerate(data):
    print('\nDataframe:', df_name[idx])
    display(df.head())
    print('\nColumn dtypes:')
    print(df.dtypes)
    print('\nNumber of null values:')
    print(df.isnull().sum())



Dataframe: pizzas


,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75
2,bbq_ckn_l,bbq_ckn,L,20.75
3,cali_ckn_s,cali_ckn,S,12.75
4,cali_ckn_m,cali_ckn,M,16.75



Column dtypes:
pizza_id          object
pizza_type_id     object
size              object
price            float64
dtype: object

Number of null values:
pizza_id         0
pizza_type_id    0
size             0
price            0
dtype: int64

Dataframe: pizza_types


,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."



Column dtypes:
pizza_type_id    object
name             object
category         object
ingredients      object
dtype: object

Number of null values:
pizza_type_id    0
name             0
category         0
ingredients      0
dtype: int64

Dataframe: orders


...


Column dtypes:
order_id     int64
date        object
time        object
dtype: object

Number of null values:
order_id    0
date        0
time        0
dtype: int64

Dataframe: order_details


,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1



Column dtypes:
order_details_id     int64
order_id             int64
pizza_id            object
quantity             int64
dtype: object

Number of null values:
order_details_id    0
order_id            0
pizza_id            0
quantity            0
dtype: int64

Dataframe: data_dictionary


,Table,Field,Description
0,orders,order_id,Unique identifier for each order placed by a t...
1,orders,date,Date the order was placed (entered into the sy...
2,orders,time,Time the order was placed (entered into the sy...
3,order_details,order_details_id,Unique identifier for each pizza placed within...
4,order_details,order_id,Foreign key that ties the details in each orde...



Column dtypes:
Table          object
Field          object
Description    object
dtype: object

Number of null values:
Table          0
Field          0
Description    0
dtype: int64


In [4]:
def anomalies(order_details, pizzas):
    anomaly = False
    for pizza in order_details['pizza_id']:
        if pizza not in pizzas['pizza_id'].unique():
            print(f'Pizza ID "{pizza}" not recognized')
            anomaly = True
    if not anomaly:
        print('No anomalies found')

anomalies(data[3], data[0])

No anomalies found


# Data transformation

In [5]:
# In this cell we create transformed csv files from the original ones

# We create a csv with the pizzas ordered in each order, instead of having each pizza in a different row
def csv_orders(orders, order_details):
    ordered_pizzas = [[] for i in range(len(orders))]
    for i in order_details['order_details_id']:
        order = order_details[order_details['order_details_id'] == i]
        for _ in range(int(order['quantity'])):
            ordered_pizzas[int(order['order_id'])-1].append(order['pizza_id'].values[0])
    orders['pizzas'] = ordered_pizzas
    orders.to_csv('TRANSFORMED/ordered_pizzas_2015.csv', index=False)
    return orders   

# This function adds to the orders dataframe the day of the week of each order
def csv_with_days(orders):
    dates = []
    days = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    for date in orders['date']:
        dates.append(days[pd.to_datetime(date, format='%d/%m/%Y').weekday()])
    orders['day'] = dates
    orders.to_csv('TRANSFORMED/ordered_pizzas_2015.csv', index=False)
    return orders

# This function creates a csv with all the ingredients and the amount of each one for each day of the week (and total week)
def csv_ingredients(pizza_types):
    all_ingredients = []
    for ingredients in pizza_types['ingredients']:
        for ingredient in ingredients.split(', '):
            ingredient = re.sub('ï¿½','', ingredient)
            if ingredient not in all_ingredients:
                all_ingredients.append(ingredient)
    ingredients_df = pd.DataFrame({'ingredient': all_ingredients, 'Monday': [0 for _ in range(len(all_ingredients))], 'Tuesday': [0 for i in range(len(all_ingredients))], 'Wednesday': [0 for i in range(len(all_ingredients))], 'Thursday': [0 for i in range(len(all_ingredients))], 'Friday': [0 for i in range(len(all_ingredients))], 'Saturday': [0 for i in range(len(all_ingredients))], 'Sunday': [0 for i in range(len(all_ingredients))], 'Total': [0 for i in range(len(all_ingredients))]})
    ingredients_df.to_csv('TRANSFORMED/ingredients_2015.csv', index=False)
    return ingredients_df



In [6]:
# We create a function that returns the pizza flavour and size of a pizza
def search_pizza(pizza_id, pizzas):
    pizza = pizzas[pizzas['pizza_id'] == pizza_id]
    return pizza['pizza_type_id'].values[0], pizza['size'].values[0]

# This function adds to the pizzas csv the amount of pizzas ordered each day of the week
# We calculate this by pizza, where we use the size as a factor 
def create_csv_with_pizzas_per_day(ordered_pizzas, pizza_types, pizzas_data, date):
    lst = [0 for i in range(len(pizza_types))]
    weigths = {'S': 1, 'M': 1.5, 'L': 2, 'XL': 2.5, 'XXL': 3}
    pizza_counts = {'Monday': lst.copy(), 'Tuesday': lst.copy(), 'Wednesday': lst.copy(), 'Thursday': lst.copy(), 'Friday': lst.copy(), 'Saturday': lst.copy(), 'Sunday': lst.copy()}
    for _, order in ordered_pizzas.iterrows():
        if pd.to_datetime(order['date'], format='%d/%m/%Y') == date:
            break
        else:
            day = order['day']
            pizzas = order['pizzas']
            for pizza in pizzas:
                pizza_flavour, size = search_pizza(pizza, pizzas_data)
                ind = pizza_types[pizza_types['pizza_type_id'] == pizza_flavour].index.values[0]
                pizza_counts[day][ind] += 1*weigths[size]
    for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        pizza_types[day] = pizza_counts[day]
    pizza_types.to_csv('TRANSFORMED/pizza_counts_2015.csv', index=False)
    return pizza_types

# This function calculates the amount of ingredients needed a specific day of the week
def ingredients_quantity(day, pizza_types, pizza_type_id, days_difference):
    aux = (pizza_types[pizza_types['pizza_type_id'] == pizza_type_id][day].values[0])
    return aux*7/days_difference

# This function predicts the ingredients needed for the following week
def predict(pizza_types, ingredients_df, days_difference):
    for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        for ingredients in pizza_types['ingredients']:
            pizza_type_id = pizza_types[pizza_types['ingredients'] == ingredients]['pizza_type_id'].values[0]
            for ingredient in ingredients.split(', '): 
                ingredient = re.sub('ï¿½','', ingredient)
                ind = ingredients_df[ingredients_df['ingredient'] == ingredient].index.values[0]   
                prediction = ingredients_quantity(day, pizza_types, pizza_type_id, days_difference)
                ingredients_df.loc[ind,[day]] += prediction
                ingredients_df.loc[ind,['Total']] += prediction
    ingredients_df.to_csv('TRANSFORMED/ingredients_2015.csv', index=False)
    return ingredients_df

# def check_ingredients(ingredients_df, pizza_types, orders, date, days_difference, pizzas_data):
#     weigths = {'S': 1, 'M': 1.5, 'L': 2, 'XL': 2.5, 'XXL': 3}
#     days = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
#     for _, order in orders.iterrows():
#         n = (pd.to_datetime(order['date'], format='%d/%m/%Y') - date).days
#         if n < 7 and n >= 0:
#             day = days[pd.to_datetime(order['date'], format='%d/%m/%Y').weekday()]
#             pizzas = order['pizzas']
#             for pizza in pizzas:
#                 pizza_flavour, size = search_pizza(pizza, pizzas_data)
#                 ingredients = pizza_types[pizza_types['pizza_type_id'] == pizza_flavour]['ingredients'].values[0]
#                 for ingredient in ingredients.split(', '): 
#                     ind = ingredients_df[ingredients_df['ingredient'] == ingredient].index.values[0]   
#                     ingredients_df.loc[ind,[day]] -= 1*weigths[size]
#                     ingredients_df.loc[ind,['Total']] -= 1*weigths[size]
#     return ingredients_df


In [7]:
date = pd.to_datetime('2015-06-15', format='%Y-%m-%d')
days_difference = (date - pd.to_datetime('2015-01-01', format='%Y-%m-%d')).days
pizzas, pizza_types, orders, order_details, data_dictionary = extract()
orders = csv_orders(orders, order_details)
orders = csv_with_days(orders)
pizza_types = create_csv_with_pizzas_per_day(orders, pizza_types, pizzas, date)
ingredients_df = csv_ingredients(pizza_types)
ingredients_df = predict(pizza_types, ingredients_df, days_difference)
print('This are the ingredients that you need to buy for the week:')
display(ingredients_df)

This are the ingredients that you need to buy for the week:


,ingredient,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Total
0,Barbecued Chicken,11.412121,11.348485,10.012121,12.154545,12.112121,12.981818,9.736364,79.757576
1,Red Peppers,72.948485,72.842424,69.660606,73.796970,85.251515,77.975758,61.472727,513.948485
2,Green Peppers,23.121212,22.103030,22.378788,23.524242,26.048485,26.663636,19.790909,163.630303
3,Tomatoes,116.942424,118.278788,112.615152,125.787879,143.754545,131.133333,101.521212,850.033333
4,Red Onions,89.281818,85.845455,83.809091,91.487879,99.739394,93.969697,74.454545,618.587879
...,...,...,...,...,...,...,...,...,...
60,Parmigiano Reggiano Cheese,8.590909,10.754545,11.009091,11.072727,11.815152,9.842424,8.081818,71.166667
61,Eggplant,3.754545,3.648485,4.221212,4.221212,5.642424,4.730303,2.990909,29.209091
62,Zucchini,9.821212,10.245455,11.072727,9.800000,13.045455,12.472727,8.548485,75.006061
63,Sun-dried Tomatoes,8.081818,7.021212,6.830303,6.978788,10.075758,8.400000,7.042424,54.430303


# Data report recommendation

In [8]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string, 'utf-8')
    return reparsed.toprettyxml(indent="    ")


In [9]:
%run quality_report.ipynb
data = {}
data['prediction_per_ingredient'] = {}
for i in range(len(ingredients_df)):
    data['prediction_per_ingredient'][re.sub(' ', '_', (ingredients_df['ingredient'][i]))] = ingredients_df['Total'][i]


In [10]:
root = ET.Element('prediction')
sub = ET.SubElement(root, 'prediction_per_ingredient')
for key, value in data['prediction_per_ingredient'].items():
    ET.SubElement(sub, key).text = str(value)

with open('data_report_2015.xml', 'a') as f:
    f.write(prettify(root))